In [1]:
import warnings
warnings.filterwarnings('ignore')

## 1. Pinecone

In [2]:
%%capture
! pip install pinecone

In [3]:
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = ""

pc = Pinecone(api_key=pinecone_api_key)

pc.create_index("llm-book", spec=ServerlessSpec("aws", "us-east-1"), dimension=768)
index = pc.Index("llm-book")

In [4]:
%%capture
! pip install datasets

In [5]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
klue_dp_train = load_dataset("klue", "dp", split="train[:100]")

embeddings = sentence_model.encode(klue_dp_train['sentence'])

In [6]:
embeddings = embeddings.tolist()

insert_data = []
for idx, (embedding, text) in enumerate(zip(embeddings, klue_dp_train['sentence'])):
    insert_data.append({
        "id": str(idx),
        "values": embedding,
        "metadata": {"text": text}
    })

In [7]:
upsert_response = index.upsert(vectors=insert_data, namespace='llm-book-sub')

In [11]:
query_response = index.query(
    namespace='llm-book-sub',
    top_k=10,
    include_values=True,
    include_metadata=True,
    vector=embeddings[0]
)
query_response

{'matches': [{'id': '0',
              'metadata': {'text': '해당 그림을 보면 디즈니 공주들이 브리트니 스피어스의 앨범이나 뮤직비디오, '
                                   '화보 속 모습을 똑같이 재연했다.'},
              'score': 1.00002146,
              'values': [-1.10073423,
                         0.220477462,
                         0.742353618,
                         0.407551259,
                         0.408452153,
                         -0.56683588,
                         0.120408647,
                         0.961137056,
                         0.12286815,
                         0.0803636238,
                         -0.269531757,
                         1.06352234,
                         0.799611747,
                         -0.652636051,
                         0.280846149,
                         0.298936129,
                         0.3499237,
                         -0.300158948,
                         -0.220105395,
                         0.143383861,
                         0.499213189,
   

In [10]:
new_text = '변경할 새로운 텍스트'
new_embedding = sentence_model.encode(new_text).tolist()

update_response = index.update(
    id='기존_문서_id',
    values=new_embedding,
    set_metadata={'text': new_text},
    namespace='llm-book-sub'
)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 28 Feb 2025 22:22:58 GMT', 'Content-Type': 'application/json', 'Content-Length': '87', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '79', 'x-pinecone-request-id': '8443623097953085659', 'x-envoy-upstream-service-time': '18', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector ID must be ASCII, but got '기존_문서_id'","details":[]}
